# Editable Install Diagnostic Notebook

This notebook helps diagnose why editable installs fail to import in Databricks notebooks.

Run each cell in order to understand what's happening.


## Step 1: Check Current Patch Status


In [ ]:
# Check if dbx-patch is installed and what patches are applied
try:
    from dbx_patch.patch_dbx import check_patch_status

    status = check_patch_status(verbose=True)
except ImportError as e:
    print(f"⚠️  dbx-patch is not installed: {e}")
    print("\nInstall it with:")
    print("  %pip install dbx-patch")

## Step 2: Check sys.path Contents


In [ ]:
import sys

print("Current sys.path entries:")
print("=" * 80)
for i, p in enumerate(sys.path, 1):
    exists = "✓" if p and __import__("os").path.exists(p) else "✗"
    print(f"{i:2d}. [{exists}] {p}")

print(f"\nTotal entries: {len(sys.path)}")

## Step 3: Detect Editable Install Paths


In [ ]:
try:
    from dbx_patch.pth_processor import find_pth_files, get_editable_install_paths, get_site_packages_dirs

    editable_paths = get_editable_install_paths()

    print(f"Detected {len(editable_paths)} editable install path(s):")
    print("=" * 80)

    if editable_paths:
        for path in sorted(editable_paths):
            in_sys_path = path in sys.path
            status = "✓ IN sys.path" if in_sys_path else "✗ NOT in sys.path"
            print(f"  [{status}] {path}")
    else:
        print("  ⚠️  No editable installs detected!")
        print("\n  This means:")
        print("  1. No packages installed with 'pip install -e .'")
        print("  2. Or .pth files are not in expected format")
        print("  3. Or site-packages directories are not accessible")

except ImportError as e:
    print(f"⚠️  Cannot import dbx_patch: {e}")
    print("Install dbx-patch first: %pip install dbx-patch")

## Step 4: Inspect .pth Files


In [ ]:
try:
    import os

    from dbx_patch.pth_processor import find_pth_files, get_site_packages_dirs

    print("Scanning site-packages for .pth files...")
    print("=" * 80)

    site_dirs = get_site_packages_dirs()
    print(f"\nFound {len(site_dirs)} site-packages director(y/ies):\n")

    total_pth_files = 0

    for site_dir in site_dirs:
        pth_files = find_pth_files(site_dir)
        total_pth_files += len(pth_files)

        print(f"📁 {site_dir}")

        if pth_files:
            for pth_file in pth_files:
                pth_name = os.path.basename(pth_file)
                print(f"   └─ {pth_name}")

                # Read and display contents
                try:
                    with open(pth_file) as f:
                        lines = [line.strip() for line in f if line.strip() and not line.startswith("#")]
                        for line in lines[:3]:  # Show first 3 lines
                            print(f"      → {line}")
                        if len(lines) > 3:
                            print(f"      ... and {len(lines) - 3} more line(s)")
                except Exception as e:
                    print(f"      ⚠️  Error reading file: {e}")
        else:
            print("   └─ (no .pth files)")

        print()

    print(f"Total .pth files found: {total_pth_files}")

except Exception as e:
    print(f"⚠️  Error: {e}")
    import traceback

    traceback.print_exc()

## Step 5: Check Import Hooks


In [ ]:
import sys

print("Import hooks in sys.meta_path:")
print("=" * 80)

for i, hook in enumerate(sys.meta_path, 1):
    hook_type = type(hook).__name__
    hook_module = type(hook).__module__
    print(f"{i}. {hook_module}.{hook_type}")

    # Check if it's a Databricks hook
    if "dbruntime" in hook_module or "WsfsPathFinder" in hook_type or "ImportHookFinder" in hook_type:
        print("   └─ ⚠️  Databricks custom hook")

print(f"\nTotal hooks: {len(sys.meta_path)}")

## Step 6: Check builtins.**import**


In [ ]:
import builtins

print("builtins.__import__ function:")
print("=" * 80)
print(f"Function: {builtins.__import__}")
print(f"Module: {builtins.__import__.__module__}")
print(f"Name: {builtins.__import__.__name__}")

if hasattr(builtins.__import__, "__self__"):
    print(f"Bound to: {builtins.__import__.__self__}")
    print("⚠️  This is a wrapped/patched import function!")
else:
    print("✓ This appears to be the standard import function")

## Step 7: Check Autoreload Allowlist


In [ ]:
try:
    from dbruntime.autoreload.file_module_utils import _AUTORELOAD_ALLOWLIST_CHECKS

    print("Autoreload allowlist checks:")
    print("=" * 80)
    print(f"Number of checks: {len(_AUTORELOAD_ALLOWLIST_CHECKS)}")

    for i, check in enumerate(_AUTORELOAD_ALLOWLIST_CHECKS, 1):
        print(f"{i}. {check}")

        # Check if it's our editable path check
        if hasattr(check, "__name__") and "editable" in check.__name__:
            print("   └─ ✓ dbx-patch editable path check FOUND!")

except ImportError:
    print("⚠️  autoreload.file_module_utils not available (may not be in Databricks)")
except Exception as e:
    print(f"⚠️  Error: {e}")

## Step 8: Apply Patches (if not already applied)


In [ ]:
# Enable logging at DEBUG level for detailed logging
import os

from dbx_patch import patch_dbx

os.environ["DBX_PATCH_ENABLED"] = "1"
os.environ["DBX_PATCH_LOG_LEVEL"] = "DEBUG"

result = patch_dbx(verbose=True)

print("\n" + "=" * 80)
print("Patch application result:")
print(f"Overall success: {result.overall_success}")
print(f"Editable paths found: {len(result.editable_paths)}")

if result.editable_paths:
    print("\nEditable paths:")
    for path in result.editable_paths:
        print(f"  - {path}")

## Step 9: Test Import with Debug Logging


In [ ]:
# Make sure debug mode is enabled
import os

os.environ["DBX_PATCH_DEBUG"] = "1"

print("Attempting to import 'testx'...")
print("=" * 80)
print("Watch for debug messages below:\n")

try:
    from testx import function1

    print("\n✓ Import succeeded!")
    print(f"function1: {function1}")
    print(f"Result: {function1()}")
except ModuleNotFoundError as e:
    print(f"\n✗ Import failed: {e}")
    print("\nThis means:")
    print("1. The module path is not in sys.path, OR")
    print("2. An import hook is blocking the import, OR")
    print("3. The module structure is incorrect (missing __init__.py)")

    # Additional diagnostics
    print("\nChecking if 'testx' source exists...")
    import os

    from dbx_patch.pth_processor import get_editable_install_paths

    for path in get_editable_install_paths():
        testx_path = os.path.join(path, "testx")
        testx_py = os.path.join(path, "testx.py")

        if os.path.isdir(testx_path):
            print(f"  Found directory: {testx_path}")
            init_py = os.path.join(testx_path, "__init__.py")
            if os.path.exists(init_py):
                print("    ✓ Has __init__.py")
            else:
                print("    ✗ Missing __init__.py!")
        elif os.path.exists(testx_py):
            print(f"  Found module file: {testx_py}")
except Exception as e:
    print(f"\n⚠️  Unexpected error: {e}")
    import traceback

    traceback.print_exc()

## Step 10: Verify Final State


In [ ]:
from dbx_patch.patch_dbx import verify_editable_installs

os.environ["DBX_PATCH_ENABLED"] = "1"
os.environ["DBX_PATCH_LOG_LEVEL"] = "INFO"

result = verify_editable_installs(verbose=True)

print("\n" + "=" * 80)
print("Final verdict:")
print(f"Status: {result.status}")
print(f"Editable paths detected: {len(result.editable_paths)}")
print(f"Paths in sys.path: {len(result.paths_in_sys_path)}")
print(f"WsfsImportHook patched: {result.wsfs_hook_patched}")
print(f"PythonPathHook patched: {result.python_path_hook_patched}")
print(f"AutoreloadHook patched: {result.autoreload_hook_patched}")

## Summary of Findings

Based on the output above:

1. **Are patches applied?** Check Step 1 output
2. **Are editable paths detected?** Check Step 3 output
3. **Are editable paths in sys.path?** Check Step 3 output
4. **What import hooks are active?** Check Step 5 output
5. **Is builtins.**import** wrapped?** Check Step 6 output
6. **Is autoreload allowlist updated?** Check Step 7 output
7. **Does import work after patching?** Check Step 9 output

## Next Steps

If import still fails after running this notebook:

1. Share the complete output from this notebook
2. Check the `.pth` file format created by `uv`
3. Verify the package structure has `__init__.py` files
4. Try manual import debugging:
   ```python
   import sys
   sys.path.insert(0, '/path/to/your/package/src')
   from testx import function1
   ```

If manual import works, the issue is with automatic path detection.
